In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [30]:
%run -i ../../zkstats/core.py

In [32]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [33]:
data_path1 = os.path.join('data1.json')
data_path2 = os.path.join('data2.json')
dummy_data_path1 = os.path.join('shared/dummy_data1.json')
dummy_data_path2 = os.path.join('shared/dummy_data2.json')

f_raw_input1 = open(data_path1, "r")
data1 = json.loads(f_raw_input1.read())["input_data"][0]
data_tensor1 = torch.reshape(torch.tensor(data1),(1, len(data1), 1))

f_raw_input2 = open(data_path2, "r")
data2 = json.loads(f_raw_input2.read())["input_data"][0]
data_tensor2 = torch.reshape(torch.tensor(data2),(1, len(data2), 1))

#  dummy data for data consumer. Just need to be the same len as private data
dummy_data1 = np.round(np.random.uniform(1,10,len(data1)),1)
json.dump({"input_data":[dummy_data1.tolist()]}, open(dummy_data_path1, 'w'))

dummy_data2 = np.round(np.random.uniform(1,10,len(data2)),1)
json.dump({"input_data":[dummy_data2.tolist()]}, open(dummy_data_path2, 'w'))

dummy_data_tensor1 = torch.reshape(torch.tensor(dummy_data1), (1, len(dummy_data1),1 ))
dummy_data_tensor2 = torch.reshape(torch.tensor(dummy_data2), (1, len(dummy_data2),1 ))
# dummy_theory_output = torch.mean(dummy_data_tensor)

dummy_combined = torch.concat((dummy_data_tensor1[0], dummy_data_tensor2[0]))


In [34]:
print(dummy_data1)
print(dummy_data2)

print(dummy_data_tensor1)
print(dummy_data_tensor2)
print("combined: ",dummy_combined )
print(torch.mean(dummy_combined))
dummy_theory_output = torch.mean(dummy_combined)
# print((torch.sum(dummy_data_tensor1)+torch.sum(dummy_data_tensor2))/(dummy_data_tensor1.size()[1]+dummy_data_tensor2.size()[1]))

[5.8 3.1 2.9]
[2.3 1.6 2.3]
tensor([[[5.8000],
         [3.1000],
         [2.9000]]], dtype=torch.float64)
tensor([[[2.3000],
         [1.6000],
         [2.3000]]], dtype=torch.float64)
combined:  tensor([[5.8000],
        [3.1000],
        [2.9000],
        [2.3000],
        [1.6000],
        [2.3000]], dtype=torch.float64)
tensor(3., dtype=torch.float64)


In [35]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,*args):
        # print(X) 
        # for data_tensor in args:
        #     print(data_tensor)
        # X_comb = torch.cat(args)
        # X_combined = X_comb.reshape(1,int(X_comb.size()[0].item()*X_comb.size()[1].item()),1)
        # print(X_combined)
        args_unpacked= tuple(arg[0] for arg in args)
        X_combined = torch.unsqueeze(torch.cat(args_unpacked),0)
        # print(X_combined.reshape(1,X_combined.size()[0]*X_combined.size()[1],1))
        # return torch.tensor(True)
        # return (X_combined.size()[1]==(6.0), self.w)
        return (torch.abs(torch.sum(X_combined)-X_combined.size()[1]*(self.w))<=torch.abs(0.01*X_combined.size()[1]*self.w), self.w)
        # return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path1, dummy_data_path2])

## Step 2
- Prover gets the model from the verifier.
- Prover overrides the `dummy_theory_output` with `theory_output`
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [36]:
# prover calculates settings, send to verifier
theory_combined = torch.concat((data_tensor1[0], data_tensor2[0]))
# print("theory combined: ", theory_combined)
theory_output = torch.mean(theory_combined)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,*args):
        args_unpacked= tuple(arg[0] for arg in args)
        X_combined = torch.unsqueeze(torch.cat(args_unpacked),0)
        # print(X_combined)
        # print(X_comb.size()[1].item())
        # X_combined = X_comb.reshape(1,int(X_comb.size()[0].item()*X_comb.size()[1].item()),1)
        # print(X_combined.size()[1]==torch.tensor(6))
        
        # print(X_combined.reshape(1,X_combined.size()[0]*X_combined.size()[1],1))
        # return torch.tensor(True)
        # return (X_combined.size()[1]==(6.0), self.w)
        return (torch.abs(torch.sum(X_combined)-X_combined.size()[1]*(self.w))<=torch.abs(0.01*X_combined.size()[1]*self.w), self.w)

prover_gen_settings([data_path1, data_path2], comb_data_path, prover_model,prover_model_path, [5], "resources", settings_path)

Theory_output:  tensor(7.5000)
==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-28,0],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":2624,"total_assignments":22,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[2624,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [37]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 0.5612740516662598 seconds
Theory output:  tensor(7.5000)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 7.5
==== Generating Proof ====
proof:  {'instances': [[[8904404911306193061, 5124839717833657588, 13990643389064988878, 325114654614251823], [204021950110864147, 12826522842573340045, 723298428150408977, 979262544395208927], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9289142259675036427, 13164462325522300906, 6491497202942540343, 2217777117322613041]]], 'proof': '2545b839b695b80dedeff42238a67d75b71d679e064262f7f434893baaafabde21dece8c374d34cf80bed70f8bc37f7b81b1158c556ad75c4d46a656b57c9732247f11cdb1c8d4142366a4e983b9d5efdc2adbde2f276f46cca4e06349bec3ad28f5ef59fa5de2f81ab17b0279c149d6c6b7c7f5379444e2388e87b026b96a8c066ec2c282d1c47b3f98200f59aaa1ef7434aeedb77bbac54368144ac81241831097ce726b6a52261adc14fe3f7d89645db2a41188d0bb5e9635966be8901b3e

In [38]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  2
prf instances:  [[[8904404911306193061, 5124839717833657588, 13990643389064988878, 325114654614251823], [204021950110864147, 12826522842573340045, 723298428150408977, 979262544395208927], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9289142259675036427, 13164462325522300906, 6491497202942540343, 2217777117322613041]]]
proof boolean:  1.0
proof result 1 : 7.5
verified
